# Exploring a file

[uproot.open](https://uproot.readthedocs.io/en/latest/opening-files.html#uproot-open) is the entry point for reading a single file.

It takes a local filename path or a remote `http://` or `root://` URL. (HTTP requires the Python [requests](https://pypi.org/project/requests/) library and XRootD requires [pyxrootd](http://xrootd.org/), both of which have to be explicitly pip-installed if you installed uproot with pip, but are automatically installed if you installed uproot with conda.)

In [14]:
import uproot

file = uproot.open("http://scikit-hep.org/uproot/examples/nesteddirs.root")
file

<ROOTDirectory b'tests/nesteddirs.root' at 0x7fa3150962e8>

[uproot.open](https://uproot.readthedocs.io/en/latest/opening-files.html#uproot-open) returns a [ROOTDirectory](https://uproot.readthedocs.io/en/latest/root-io.html#uproot-rootio-rootdirectory), which behaves like a Python dict; it has `keys()`, `values()`, and key-value access with square brackets.

In [15]:
file.keys()

[b'one;1', b'three;1']

In [16]:
file["one"]

<ROOTDirectory b'one' at 0x7fa3150967f0>

Subdirectories also have type [ROOTDirectory](https://uproot.readthedocs.io/en/latest/root-io.html#uproot-rootio-rootdirectory), so they behave like Python dicts, too.

In [17]:
file["one"].keys()

[b'two;1', b'tree;1']

In [18]:
file["one"].values()

[<ROOTDirectory b'two' at 0x7fa3150d0128>, <TTree b'tree' at 0x7fa315096dd8>]

**What's the `b` before each object name?** Python 3 distinguishes between bytestrings and encoded strings. ROOT object names have no encoding, such as Latin-1 or Unicode, so uproot presents them as raw bytestrings. However, if you enter a Python string (no `b`) and it matches an object name (interpreted as plain ASCII), it will count as a match, as `"one"` does above.

**What's the `;1` after each object name?** ROOT objects are versioned with a "cycle number." If multiple objects are written to the ROOT file with the same name, they will have different cycle numbers, with the largest value being last. If you don't specify a cycle number, you'll get the latest one.

This file is deeply nested, so while we could find the TTree with

In [19]:
file["one"]["two"]["tree"]

<TTree b'tree' at 0x7fa3150d0898>

we can also find it using a directory path, with slashes.

In [20]:
file["one/two/tree"]

<TTree b'tree' at 0x7fa3150d09b0>

Here are a few more tricks for finding your way around a file:

   * the `keys()`, `values()`, and `items()` methods have `allkeys()`, `allvalues()`, `allitems()` variants that recursively search through all subdirectories;
   * all of these functions can be filtered by name or class: see [ROOTDirectory.keys](https://uproot.readthedocs.io/en/latest/root-io.html#uproot.rootio.ROOTDirectory.keys).

Here's how you would search the subdirectories to find all TTrees:

In [21]:
file.allkeys(filterclass=lambda cls: issubclass(cls, uproot.tree.TTreeMethods))

[b'one/two/tree;1', b'one/tree;1', b'three/tree;1']

Or get a Python dict of them:

In [23]:
all_ttrees = dict(file.allitems(filterclass=lambda cls: issubclass(cls, uproot.tree.TTreeMethods)))
all_ttrees

{b'one/two/tree;1': <TTree b'tree' at 0x7fa31508ceb8>,
 b'one/tree;1': <TTree b'tree' at 0x7fa31508cfd0>,
 b'three/tree;1': <TTree b'tree' at 0x7fa31508ca20>}

Be careful: Python 3 is not as forgiving about matching key names. `all_ttrees` is a plain Python dict, so the key must be a bytestring and must include the cycle number.

In [24]:
all_ttrees[b"one/two/tree;1"]

<TTree b'tree' at 0x7fa31508ceb8>

## Exploring a TTree

TTrees are special objects in ROOT files: they contain most of the physics data. Uproot presents TTrees as subclasses of [uproot.tree.TTreeMethods](https://uproot.readthedocs.io/en/latest/ttree-handling.html#uproot-tree-ttreemethods).

(**Why subclass?** Different ROOT files can have different versions of a class, so uproot generates Python classes to fit the data, as needed. All TTrees inherit from [uproot.tree.TTreeMethods](https://uproot.readthedocs.io/en/latest/ttree-handling.html#uproot-tree-ttreemethods) so that they get the same data-reading methods.)

In [26]:
events = uproot.open("http://scikit-hep.org/uproot/examples/Zmumu.root")["events"]
events

<TTree b'events' at 0x7fa314ff9f60>

Although [uproot.tree.TTreeMethods](https://uproot.readthedocs.io/en/latest/ttree-handling.html#uproot-tree-ttreemethods) objects behave like Python dicts of [uproot.tree.TBranchMethods](https://uproot.readthedocs.io/en/latest/ttree-handling.html#uproot-tree-tbranchmethods) objects, the easiest way to browse a TTree is by calling its `show()` method, which prints the branches and their interpretations as arrays.

In [27]:
events.show()

Type                       (no streamer)              asstring()
Run                        (no streamer)              asdtype('>i4')
Event                      (no streamer)              asdtype('>i4')
E1                         (no streamer)              asdtype('>f8')
px1                        (no streamer)              asdtype('>f8')
py1                        (no streamer)              asdtype('>f8')
pz1                        (no streamer)              asdtype('>f8')
pt1                        (no streamer)              asdtype('>f8')
eta1                       (no streamer)              asdtype('>f8')
phi1                       (no streamer)              asdtype('>f8')
Q1                         (no streamer)              asdtype('>i4')
E2                         (no streamer)              asdtype('>f8')
px2                        (no streamer)              asdtype('>f8')
py2                        (no streamer)              asdtype('>f8')
pz2                        (no streame

The bulk data have not been read yet. There are many ways to do that:

   * select a TBranch and call [TBranchMethods.array](https://uproot.readthedocs.io/en/latest/ttree-handling.html#id11);
   * call [TTreeMethods.array](https://uproot.readthedocs.io/en/latest/ttree-handling.html#array) directly from the TTree object;
   * call [TTreeMethods.arrays](https://uproot.readthedocs.io/en/latest/ttree-handling.html#arrays) to get several arrays at a time;
   * call [TBranch.lazyarray](https://uproot.readthedocs.io/en/latest/ttree-handling.html#id13), [TTreeMethods.lazyarray](https://uproot.readthedocs.io/en/latest/ttree-handling.html#lazyarray), [TTreeMethods.lazyarrays](https://uproot.readthedocs.io/en/latest/ttree-handling.html#lazyarrays), or [uproot.lazyarrays](https://uproot.readthedocs.io/en/latest/opening-files.html#uproot-lazyarray-and-lazyarrays) to get array-like objects that read on demand;
   * call [TTreeMethods.iterate](https://uproot.readthedocs.io/en/latest/ttree-handling.html#iterate) or [uproot.iterate](https://uproot.readthedocs.io/en/latest/opening-files.html#uproot-iterate) to explicitly iterate over chunks of data (to avoid reading more than would fit into memory);
   * call [TTreeMethods.pandas](https://uproot.readthedocs.io/en/latest/ttree-handling.html#id7) or [uproot.pandas.iterate](https://uproot.readthedocs.io/en/latest/opening-files.html#uproot-pandas-iterate) to get Pandas DataFrames ([Pandas](https://pandas.pydata.org/) must be installed).